# Importing Dependencies

In [42]:
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re 
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 
print('Dependencies Loaded Successfully')

Dependencies Loaded Successfully


In [43]:
# Importing the stopwords
import nltk
print(stopwords.words('english')) #English Stopwords

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Importing Data 

In [44]:
news_df = pd.read_csv('data/train.csv')
news_test = pd.read_csv('data/test.csv')

In [45]:
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [46]:
news_test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


#### 1 : Fake News, 0 : Real News

# Basic Data Analysis

In [47]:
news_df.shape

(20800, 5)

In [48]:
news_test.shape

(5200, 4)

In [49]:
print(news_df['label'].value_counts())

1    10413
0    10387
Name: label, dtype: int64


In [50]:
# Checking for missing values in the dataset
news_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [51]:
news_test.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [52]:
# Replacing the null values with emtpy strings 
news_df = news_df.fillna('')

In [53]:
news_test = news_test.fillna('')

In [54]:
# Checking for missing values again
news_df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [55]:
news_test.isnull().sum()

id        0
title     0
author    0
text      0
dtype: int64

# Classification Plan :
#### To use '*Title*' and '*Author*' data columns to make predictions

In [56]:
#merging the author name and news title 
news_df['total'] = news_df['author']+' '+news_df['title']
news_test['total'] = news_test['author']+' '+news_test['title']

In [57]:
print(news_df['total'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: total, Length: 20800, dtype: object


In [58]:
print(news_test['total'])

0       David Streitfeld Specter of Trump Loosens Tong...
1        Russian warships ready to strike terrorists n...
2       Common Dreams #NoDAPL: Native American Leaders...
3       Daniel Victor Tim Tebow Will Attempt Another C...
4       Truth Broadcast Network Keiser Report: Meme Wa...
                              ...                        
5195    Jody Rosen The Bangladeshi Traffic Jam That Ne...
5196    Sheryl Gay Stolberg John Kasich Signs One Abor...
5197    Mike McPhate California Today: What, Exactly, ...
5198     300 US Marines To Be Deployed To Russian Bord...
5199    Teddy Wayne Awkward Sex, Onscreen and Off - Th...
Name: total, Length: 5200, dtype: object


In [59]:
# separating the data & label
X = news_df.drop(columns='label', axis=1)
Y = news_df['label']

In [60]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [61]:
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


# Stemming Process 
#### In short : Reducing a word to its root word, removing prefix and suffix 

In [62]:
port_stem = PorterStemmer()

#### Defining the stemmer function

In [63]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [64]:
news_df['total'] = news_df['total'].apply(stemming)
news_test['total'] = news_test['total'].apply(stemming)

In [65]:
news_df['total'].head()

0    darrel lucu hous dem aid even see comey letter...
1    daniel j flynn flynn hillari clinton big woman...
2               consortiumnew com truth might get fire
3    jessica purkiss civilian kill singl us airstri...
4    howard portnoy iranian woman jail fiction unpu...
Name: total, dtype: object

In [66]:
news_test['total'].head()

0    david streitfeld specter trump loosen tongu pu...
1    russian warship readi strike terrorist near al...
2    common dream nodapl nativ american leader vow ...
3    daniel victor tim tebow attempt anoth comeback...
4     truth broadcast network keiser report meme war e
Name: total, dtype: object

In [67]:
# Seperating the data and the label 

X = news_df['total'].values
Y = news_df['label'].values
Y

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [68]:
submit = pd.read_csv('data/submit.csv')

In [69]:
X_test = news_test['total'].values
Y_test = submit['label'].values
Y_test

array([0, 1, 0, ..., 0, 1, 0], dtype=int64)

# Vectorizing the content data

In [70]:
# converting the textual data into numerical data 
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [71]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X_test)
X_test = vectorizer.transform(X_test)

# Training the Logistic Regression Model

In [72]:
model = LogisticRegression()

In [73]:
model.fit(X, Y)

LogisticRegression()

# Model Evaluation

#### A. On Training Data

In [74]:
X_train_prediction = model.predict(X)

In [75]:
training_data_accuracy = accuracy_score(X_train_prediction, Y)

In [76]:
 print('Accuracy score of training data :', training_data_accuracy)

Accuracy score of training data : 0.9883173076923077


In [83]:
X_test_prediction = model.predict(X_test)

ValueError: X has 9142 features per sample; expecting 17128

In [ ]:
testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
 print('Accuracy score of training data :', testing_data_accuracy)